# 2. Importar las librerías necesarias:

In [182]:
import pandas as pd
import numpy as np

# 3. Cargar los datos (lectura y procesamiento de datos - Pandas):

In [183]:


df_sales = pd.read_csv("/workspace/sales.csv")
df_inventory = pd.read_csv("/workspace/inventories.csv")
df_satisfaction = pd.read_csv("/workspace/satisfaction.csv")


In [184]:
df_sales.dropna(inplace=True)
df_inventory.dropna(inplace=True)
df_satisfaction.dropna(inplace=True)


# 4. Exploración de datos (Pandas)

In [185]:
# Calcula las ventas totales por producto y por tienda.

df_sales[["ID_Tienda", "Producto", "Cantidad_Vendida"]].groupby(["ID_Tienda", "Producto"]).sum().reset_index()

,ID_Tienda,Producto,Cantidad_Vendida
0,1,Producto A,20
1,1,Producto B,15
2,2,Producto A,30
3,2,Producto C,25
4,3,Producto A,10
5,3,Producto B,40
6,4,Producto A,25
7,4,Producto C,35
8,5,Producto B,20
9,5,Producto C,30


In [186]:
# Calcula los ingresos totales por tienda, multiplicando la cantidad vendida por el precio unitario.
df_sales["Total_Vendido"] = df_sales["Cantidad_Vendida"] * df_sales["Precio_Unitario"]
df_sales[["ID_Tienda", "Total_Vendido"]].groupby("ID_Tienda").sum().reset_index()

,ID_Tienda,Total_Vendido
0,1,5000
1,2,10500
2,3,9000
3,4,13000
4,5,13000


In [187]:
# Genera un resumen estadístico de las ventas utilizando el método describe() para obtener la media, mediana y otras métricas clave.
df_sales.describe()

,ID_Tienda,Cantidad_Vendida,Precio_Unitario,Total_Vendido
count,10.000000,10.000000,10.000000,10.000000
mean,3.000000,25.000000,190.000000,5050.000000
std,1.490712,9.128709,87.559504,3361.960407
min,1.000000,10.000000,100.000000,1000.000000
25%,2.000000,20.000000,100.000000,2625.000000
50%,3.000000,25.000000,200.000000,3500.000000
75%,4.000000,30.000000,275.000000,7875.000000
max,5.000000,40.000000,300.000000,10500.000000


In [188]:
#Si los productos están clasificados por categorías, calcula el promedio de ventas por tienda y categoría de productos.
df_sales[["ID_Tienda","Producto","Total_Vendido"]].groupby(["ID_Tienda","Producto"]).mean().reset_index()

,ID_Tienda,Producto,Total_Vendido
0,1,Producto A,2000.0
1,1,Producto B,3000.0
2,2,Producto A,3000.0
3,2,Producto C,7500.0
4,3,Producto A,1000.0
5,3,Producto B,8000.0
6,4,Producto A,2500.0
7,4,Producto C,10500.0
8,5,Producto B,4000.0
9,5,Producto C,9000.0


# 5. Análisis de inventarios (Pandas)

In [189]:
#Calcula la rotación de inventarios para cada tienda. Esto se hace dividiendo las ventas totales por el stock disponible de cada producto.
df_rotacion_inventario = df_sales[["ID_Tienda","Producto","Total_Vendido"]].merge(df_inventory[["ID_Tienda","Producto", "Stock_Disponible"]])
df_rotacion_inventario["Rotacion_Inventario"] = df_rotacion_inventario["Total_Vendido"] / df_rotacion_inventario["Stock_Disponible"]
df_rotacion_inventario

,ID_Tienda,Producto,Total_Vendido,Stock_Disponible,Rotacion_Inventario
0,1,Producto A,2000,50,40.000000
1,1,Producto B,3000,40,75.000000
2,2,Producto A,3000,60,50.000000
3,2,Producto C,7500,45,166.666667
4,3,Producto A,1000,30,33.333333
5,3,Producto B,8000,80,100.000000
6,4,Producto C,10500,70,150.000000
7,4,Producto A,2500,50,50.000000
8,5,Producto B,4000,40,100.000000
9,5,Producto C,9000,60,150.000000


In [190]:
#Calcula la rotación de inventarios para cada tienda. Esto se hace dividiendo las ventas totales por el stock disponible de cada producto.
df_inventory = df_inventory.merge(df_sales[["ID_Tienda","Producto","Cantidad_Vendida"]])
df_inventory["Rotacion_Inventario"] = df_inventory["Cantidad_Vendida"] / df_inventory["Stock_Disponible"]
df_inventory.drop("Cantidad_Vendida", axis=1, inplace=True)
df_inventory["Porcentaje_Rotacion"] = df_inventory["Rotacion_Inventario"] * 100
df_inventory[df_inventory["Porcentaje_Rotacion"]<10]

,ID_Tienda,Producto,Stock_Disponible,Fecha_Actualización,Rotacion_Inventario,Porcentaje_Rotacion


# 6. Satisfacción del cliente (Pandas)

In [191]:
# Filtra las tiendas con niveles bajos de satisfacción (< 60%) y haz recomendaciones para mejorar el rendimiento de estas tiendas.
df_sales = df_sales.merge(df_satisfaction[["ID_Tienda", "Satisfacción_Promedio"]], how="left")
df_sales[df_sales["Satisfacción_Promedio"]<60]

# RECOMENDACIONES
# Esta es la segunda tienda que más vende en comparación al resto de tiendas.
# Los volumenes de ventas parece tener una relación con la satisfacción promedio. Se observa que las tiendas con menos ventas tienen una satisfacción promedio mayor. Esto puede sugerir que la satisfacción de venta está relacionada al volumen de personas que entran a las tienda.
# Como recomendación se puede capacitar al personal para mantener una atención equivalente tanto si el volumen de ventas es bajo como alto.

,ID_Tienda,Producto,Cantidad_Vendida,Precio_Unitario,Fecha_Venta,Total_Vendido,Satisfacción_Promedio
8,5,Producto B,20,200,2023-01-13,4000,55
9,5,Producto C,30,300,2023-01-14,9000,55


# 7. Operaciones con Numpy

In [192]:
#Usar Numpy para realizar los siguientes cálculos sobre las ventas:
#Mediana de las ventas totales.
#Desviación estándar de las ventas totales.
print(f"La mediana calculado por numpy es {np.median(df_sales['Total_Vendido'].values)}")
print(f"La desviación estandar calculado por numpy es {np.std(df_sales['Total_Vendido'].values)}")

La mediana calculado por numpy es 3500.0
La desviación estandar calculado por numpy es 3189.4356867634124


In [193]:
#Genera arrays aleatorios utilizando la biblioteca Numpy para simular proyecciones de ventas futuras.
np.random.seed(0)
id_tiendas = [1,2,3,4,5]
productos = ["Productos A","Productos B","Productos C"]
fechas_venta = ["2023-01-15","2023-01-16","2023-01-17","2023-01-18","2023-01-19"]
precios_producto = {"Productos A": 100,"Productos B":200,"Productos C":300}

for fecha in fechas_venta:
  cantidad_vendida = np.random.randint(10,40,1)[0]
  producto = np.random.choice(productos)
  df_sale = pd.DataFrame([[
    np.random.choice(id_tiendas),
    producto,
    cantidad_vendida,
    precios_producto[producto],
    fecha,
    cantidad_vendida * precios_producto[producto],
    0
  ]], columns=["ID_Tienda","Producto","Cantidad_Vendida","Precio_Unitario","Fecha_Venta","Total_Vendido","Satisfacción_Promedio"])
  df_sales = pd.concat([df_sales, df_sale])
df_sales.reset_index(inplace=True, drop=True)

In [194]:
print(f"La mediana calculado por numpy es {np.median(df_sales['Total_Vendido'].values)}")
print(f"La desviación estandar calculado por numpy es {np.std(df_sales['Total_Vendido'].values)}")

La mediana calculado por numpy es 4000.0
La desviación estandar calculado por numpy es 3174.8595769471967
